<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Import-modules" data-toc-modified-id="Import-modules-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import modules</a></span></li><li><span><a href="#Get-dataverse-info" data-toc-modified-id="Get-dataverse-info-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get dataverse info</a></span></li><li><span><a href="#Get-dataset-info" data-toc-modified-id="Get-dataset-info-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Get dataset info</a></span></li></ul></li></ul></div>

## Plan

For chosen dataverse:
    - Show dataverse info:
        - Show number of subdataverses, if any
        - Show whether or not the (sub)dataverse has a description or tagline
        - List metadatablocks of each (sub)dataverse
        - List the facets set for each (sub)dataverse
        - Verify that contact email address is valid for each (sub)dataverse
        (see https://medium.com/@arjunsinghy96/verify-emails-over-socks-proxy-using-python-5589cb75c405
        and https://github.com/Gardener-gg/email-verifier)
    - Show dataset info
        - Show number of datasets
        - Show date of first published dataset
        - Show date of most recently published or updated dataset
        - Show average age of datasets
        - Show average number of dataset versions
        - Metadata (of latest published version of each dataset):
            - Show average number of characters in the dataset descriptions
                - List datasets with fewer than a certain number of characters in their descriptions
            - List datasets with CC0 or Terms of Use metadata
                - Versus number of datasets with no CC0 or TOU metadata
                - List datasets with no CC0 or TOU metadata
            - Show number of files that have no description metadata
                - If a certain percentage of datasets have 1 or more files with no descriptions, list those datasets
            - Related publication metadata
                - Show number of datasets with related publication metadata
                    - List datasets with no related publication metadata
                - Show number of datasets with no PID in related publication metadata
                    - List datasets with no PID in related publication metadata
            - Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse
        - Data
            - List count of each unique file format
            - Show number of datasets with no files
                - List datasets that have no files
            - Show number of datasets with 1 or more uningested tabular files
                - List datasets that contain 1 or more uningested tabular files
            - Show number of datasets with 1 or more restricted files
        - Contact emails
            - Get number of datasets that have a contact email address that's different email from the dataverse contact email address
            - Get unique list of contact email addresses and check if they're valid
            - Show any datasets that have no valid email addresses

## Code

### Import modules

In [127]:
import numpy as np
import pandas as pd
import requests

def improved_get(_dict, path, default=None):
    for key in path.split('.'):
        try:
            _dict = _dict[key]
        except KeyError:
            return default
    return _dict

### Get dataverse info

In [128]:
# Get dataverse alias from user - return error if there's no alias or if alias is the Root dataverse
# server = 'https://demo.dataverse.org'
# dataverseAlias = 'sefsef'
server = 'https://dataverse.harvard.edu'
dataverseAlias = 'covid-rt'


In [129]:
# Get info from that dataverse: whether or not the dataverse has a description and/or tagline, metadatablocks enabled, facets enabled, validate contact email
dataverseInfoApi = '%s/api/dataverses/%s' % (server, dataverseAlias)
response = requests.get(dataverseInfoApi)
dataverseMetadata = response.json()

In [130]:
if dataverseMetadata['status'] == 'ERROR':
    print('No dataverse found. Is the dataverse published on Harvard Dataverse?')
elif dataverseMetadata['status'] == 'OK':
    dataverseDescription = improved_get(dataverseMetadata, 'data.description')
    if dataverseDescription is not None:
        print('Dataverse has a description')
    else:
        print('Dataverse has no description')

    tagline = improved_get(dataverseMetadata, 'data.theme.tagline')
    if tagline is not None:
        print('Dataverse has a tagline')
    else:
        print('Dataverse has no tagline')

#     contactEmails = []
#     for contact in dataverseMetadata['data']['dataverseContacts']:
#         contactEmails.append(contact['contactEmail'])
#     print(contactEmails)

    dataverseFacetsApi = '%s/api/dataverses/%s/facets' % (server, dataverseAlias)
    response = requests.get(dataverseFacetsApi)
    dataverseFacets = response.json()
    facets = []
    for facet in dataverseFacets['data']:
        facets.append(facet)
    print('Number of search facets used: %s' % (len(facets)))    

    dataverseMetadatablocksList = []
    dataverseMetadatablocksApi = '%s/api/dataverses/%s/metadatablocks' % (server, dataverseAlias)
    response = requests.get(dataverseMetadatablocksApi)
    dataverseMetadatablocks = response.json()
    for dataverseMetadatablock in dataverseMetadatablocks['data']:
        dataverseMetadatablock = dataverseMetadatablock['name']
        dataverseMetadatablocksList.append(dataverseMetadatablock)
    print('Number of metadatablocks enabled (in addition to Citation): %s' % (len(dataverseMetadatablocksList) - 1))


Dataverse has a description
Dataverse has no tagline
Number of search facets used: 3
Number of metadatablocks enabled (in addition to Citation): 0
['citation']


### Get dataset info

In [131]:
# Get PIDs of all published datasets and their files in that dataverse
getContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseAlias)
response = requests.get(getContentsApi)
dataverseContents = response.json()


In [132]:
datasetPIDs = []
for item in dataverseContents['data']:
    if item['type'] == 'dataset':
        datasetPID = item['persistentUrl'].replace('https://doi.org/', 'doi:')
        datasetPIDs.append(datasetPID)
print('Number of datasets: %s' % (len(datasetPIDs)))


Number of datasets: 16


Create a dataframe for dataset info: date of publication, the release date of the latest version, number of versions

_Getting this info can be slow. For example, getting the info of ~375 datasets might take 45 min_

In [137]:
# List of file types that Dataverse can convert to .tab files
uningestedFileTypes = ['application/x-rlang-transport', 'application/x-stata-13', 'application/x-spss-por',
                      'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'text/csv', 'text/tsv',
                      'application/x-spss-sav', 'text/comma-separated-values', 'application/x-stata',
                      'application/x-stata-14']

rowList = []
datasetCount = 0
relPubCount = 0
relPubPIDCount = 0
for datasetPID in datasetPIDs:
    getAllVersionsApi = '%s/api/datasets/:persistentId/versions?persistentId=%s' % (server, datasetPID)
    response = requests.get(getAllVersionsApi)
    datasetVersions = response.json()
    
    # Get only datasets with metadata (exclude responses with no values in 'data' key, e.g. deaccessioned datasets)
    if datasetVersions['status'] == 'OK' and len(datasetVersions['data']) > 0:

        # Get index location of first dataset version
        firstVersion = len(datasetVersions['data']) - 1

        # Get length of description text
        descriptionLength = 0
        for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'dsDescription':
                for i in field['value']:
                    descriptionLength = descriptionLength + len(i['dsDescriptionValue']['value'])

        # See whether CC0 or Terms of Use metadata exists
        license = datasetVersions['data'][0]['license']
        try:
            termsOfUse = datasetVersions['data'][0]['termsOfUse']
            termsOfUse = True
        except KeyError:
            termsOfUse = False

        if license != 'CC0' and termsOfUse == False:
            termsExist = False
        else:
            termsExist = True

        # Get info about related publication metadata
        for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'publication':
                for value in field['value']:
                    relPubCount += 1
                    try:
                        publicationIDType = value['publicationIDType']
                        publicationIDNumber = value['publicationIDNumber']
                        relPubPIDCount += 1
                    except KeyError:
                        relPubPIDCount = relPubPIDCount
            else:
                relPubCount = relPubCount
        
        # Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse
        # dataverseMetadatablocks List of dataverse's enabled metadatablocks
        unusedMetadatablocks = []
        for metadatablock in dataverseMetadatablocksList:
            try:
                fieldCount = len(datasetVersions['data'][0]['metadataBlocks'][metadatablock]['fields'])
                if fieldCount == 0:
                    unusedMetadatablocks.append(metadatablock)
            except KeyError:
                unusedMetadatablocks.append(metadatablock)
        if len(unusedMetadatablocks) == 0:
            unusedMetadatablocks = ''

        # Get number of files
        numberOfFiles = len(datasetVersions['data'][0]['files'])

        # Get file info
        noFileDescriptionCount = 0
        contentType = []
        uningestedTabFilesCount = 0
        restrictedFilesCount = 0
        for file in datasetVersions['data'][0]['files']:            
            if 'description' in file:
                noFileDescriptionCount = noFileDescriptionCount
            else:
                noFileDescriptionCount += 1
            contentType.append(file['dataFile']['contentType'])
            if file['restricted'] == True:
                restrictedFilesCount += 1
            if file['dataFile']['contentType'] in uningestedFileTypes:
                uningestedTabFilesCount += 1

        if len(contentType) == 0:
            uniqueContentTypes = 'NA'
        else:
            uniqueContentTypes = repr(set(contentType)).lstrip('{').rstrip('}')

        # Create dictionary
        newRow = {'datasetPID': datasetPID,
                  'numberOfVersions': len(datasetVersions['data']),
                  'publicationDate': datasetVersions['data'][firstVersion]['releaseTime'],
                  'latestReleaseDate': datasetVersions['data'][0]['releaseTime'],
                  'descriptionLenth': descriptionLength,
                  'termsExist': termsExist,
                  'relPubCount': relPubCount,
                  'relPubPIDCount': relPubPIDCount,
                  'unusedMetadatablocks': unusedMetadatablocks,
                  'numberOfFiles': numberOfFiles,
                  'noFileDescriptionCount': noFileDescriptionCount,
                  'uniqueContentTypes': uniqueContentTypes,
                  'uningestedTabFilesCount': uningestedTabFilesCount,
                  'restrictedFilesCount': restrictedFilesCount
                 }
        rowList.append(dict(newRow))
        datasetCount += 1
        print('%s of %s (%s)' % (datasetCount, len(datasetPIDs), datasetPID))
        
if len(datasetPIDs) != datasetCount:
    print('The metadata of %s dataset(s) could not be retrieved' % (len(datasetPIDs) - datasetCount))


1 of 16 (doi:10.7910/DVN/XPULZP)
2 of 16 (doi:10.7910/DVN/8DUSHZ)
3 of 16 (doi:10.7910/DVN/PRP6CY)
4 of 16 (doi:10.7910/DVN/LNMJYJ)
5 of 16 (doi:10.7910/DVN/GI8EVP)
6 of 16 (doi:10.7910/DVN/METDW2)
7 of 16 (doi:10.7910/DVN/B4UO2L)
8 of 16 (doi:10.7910/DVN/RBZVJE)
9 of 16 (doi:10.7910/DVN/TTLQRN)
10 of 16 (doi:10.7910/DVN/A12ADQ)
11 of 16 (doi:10.7910/DVN/K2PXLV)
12 of 16 (doi:10.7910/DVN/BZ7FPH)
13 of 16 (doi:10.7910/DVN/CCE4XT)
14 of 16 (doi:10.7910/DVN/QVWUJ5)
15 of 16 (doi:10.7910/DVN/S07EZB)
16 of 16 (doi:10.7910/DVN/2CNKZJ)


In [138]:
df = pd.DataFrame(rowList)               
df

,datasetPID,numberOfVersions,publicationDate,latestReleaseDate,descriptionLenth,termsExist,relPubCount,relPubPIDCount,unusedMetadatablocks,numberOfFiles,noFileDescriptionCount,uniqueContentTypes,uningestedTabFilesCount,restrictedFilesCount
0,doi:10.7910/DVN/XPULZP,2,2020-10-23T16:52:04Z,2020-10-26T09:46:35Z,426,True,0,0,,10,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
1,doi:10.7910/DVN/8DUSHZ,5,2020-10-23T16:51:46Z,2020-10-26T09:44:38Z,409,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
2,doi:10.7910/DVN/PRP6CY,5,2020-10-23T16:51:29Z,2020-10-26T08:51:46Z,409,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
3,doi:10.7910/DVN/LNMJYJ,5,2020-10-23T16:51:12Z,2020-10-26T08:50:46Z,411,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
4,doi:10.7910/DVN/GI8EVP,4,2020-10-23T16:50:56Z,2020-10-26T06:48:51Z,412,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
5,doi:10.7910/DVN/METDW2,5,2020-10-23T16:50:27Z,2020-10-26T06:46:22Z,410,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
6,doi:10.7910/DVN/B4UO2L,4,2020-10-23T16:50:13Z,2020-10-26T06:07:09Z,411,True,0,0,,15,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
7,doi:10.7910/DVN/RBZVJE,12,2020-10-23T16:49:50Z,2020-10-27T02:49:01Z,400,True,0,0,,8,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
8,doi:10.7910/DVN/TTLQRN,9,2020-10-23T16:49:34Z,2020-10-26T04:04:12Z,400,True,0,0,,12,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
9,doi:10.7910/DVN/A12ADQ,5,2020-10-23T16:49:19Z,2020-10-26T04:01:35Z,400,True,0,0,,12,0,"'text/tab-separated-values', 'image/png', 'app...",0,0
